In [9]:
from itertools import product

import os
import pandas as pd


base_path = "/scratch/gpfs/jp7427/out/isotropic-decaying-turbulence2/128x128/cupy/"

In [10]:
# Loop parameters
M_max_values = [0.01, 0.1, 1, 10, 50]

seeds = range(1, 31)

configs = [
    "MUSCL-Hancock",
    "ZS3",
    "ZS7",
    "ZS3t",
    "ZS7t",
    "MM3-3/rtol_1e-2_atol_1e-14",
    "MM7-3/rtol_1e-2_atol_1e-14",
    "MM3-3/rtol_1e-1_atol_1e-14",
    "MM7-3/rtol_1e-1_atol_1e-14",
    "MM3-3/rtol_1e0_atol_1e-14",
    "MM7-3/rtol_1e0_atol_1e-14",
    "MM3-3/rtol_1e1_atol_1e-14",
    "MM7-3/rtol_1e1_atol_1e-14",
]

# read simulation outputs
data = []
for config, M_max in product(configs, M_max_values):
    entry = dict(config=config, M_max=M_max, failed=0)
    for seed in seeds:
        if M_max < 1 and seed > 1:
            continue

        sim_path = f"{base_path}{config}/M_max_{M_max}/seed_{seed:02d}/"

        if not os.path.exists(sim_path):
            raise FileNotFoundError(sim_path)
        elif os.path.exists(sim_path + "error.txt"):
            entry["failed"] += 1
        elif os.path.exists(sim_path + "timings.txt"):
            continue
        else:
            raise FileNotFoundError(f"error.txt or timings.txt in {sim_path}")
    data.append(entry)

df = pd.DataFrame(data)

FileNotFoundError: error.txt or timings.txt in /scratch/gpfs/jp7427/out/isotropic-decaying-turbulence2/128x128/cupy/MM7-3/rtol_1e-2_atol_1e-14/M_max_0.01/seed_01/

In [11]:
df.pivot(index="M_max", columns="config", values="failed").loc[
    :,
    configs,
].sort_index(ascending=True)

config,MUSCL-Hancock,ZS3,ZS7,ZS3t,ZS7t,MM3-3/rtol_1e-2_atol_1e-14,MM7-3/rtol_1e-2_atol_1e-14,MM3-3/rtol_1e-1_atol_1e-14,MM7-3/rtol_1e-1_atol_1e-14,MM3-3/rtol_1e0_atol_1e-14,MM7-3/rtol_1e0_atol_1e-14,MM3-3/rtol_1e1_atol_1e-14,MM7-3/rtol_1e1_atol_1e-14
M_max,,,,,,,,,,,,,
0.01,0,0,0,0,0,0,0,0,0,0,0,0,0
0.10,0,0,0,0,0,0,0,0,0,0,0,0,0
1.00,0,0,0,0,0,0,0,0,0,5,6,7,30
10.00,0,0,0,30,30,0,0,0,0,30,30,30,30
50.00,0,0,0,30,30,0,1,0,0,30,30,30,30


In [4]:
df_to_table = df[df.M_max >= 1]


df_to_table["config"] = pd.Categorical(
    df_to_table["config"], categories=configs, ordered=True
)

print(df_to_table.pivot(index="config", columns="M_max", values="failed").to_latex())

\begin{tabular}{lrrrrrr}
\toprule
M_max & 1.000000 & 10.000000 & 20.000000 & 30.000000 & 40.000000 & 50.000000 \\
config &  &  &  &  &  &  \\
\midrule
p0 & 0 & 0 & 0 & 0 & 0 & 0 \\
MUSCL-Hancock & 0 & 0 & 0 & 0 & 0 & 0 \\
ZS3 & 0 & 0 & 0 & 0 & 0 & 0 \\
ZS7 & 0 & 0 & 0 & 0 & 0 & 0 \\
ZS3t & 0 & 30 & 30 & 30 & 30 & 30 \\
ZS7t & 0 & 30 & 30 & 30 & 30 & 30 \\
MM3-3/rtol_1e-2_atol_1e-14 & 0 & 0 & 0 & 0 & 0 & 0 \\
MM7-3/rtol_1e-2_atol_1e-14 & 0 & 0 & 0 & 0 & 0 & 0 \\
MM3-3/rtol_1e-1_atol_1e-14 & 0 & 0 & 0 & 0 & 0 & 0 \\
MM7-3/rtol_1e-1_atol_1e-14 & 0 & 0 & 0 & 0 & 0 & 0 \\
MM3-3/rtol_1e0_atol_1e-14 & 0 & 30 & 30 & 30 & 30 & 30 \\
MM7-3/rtol_1e0_atol_1e-14 & 1 & 30 & 30 & 30 & 30 & 30 \\
MM3-3/rtol_1e1_atol_1e-14 & 11 & 30 & 30 & 30 & 30 & 30 \\
MM7-3/rtol_1e1_atol_1e-14 & 29 & 30 & 30 & 30 & 30 & 30 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1057696/1095294678.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_table["config"] = pd.Categorical(
